In [34]:
import pandas as pd

SEED= 1123445635

df= pd.read_json('../../data/nytimes_dataset_full.json')
df=df.rename(columns={'section':'labels'})
len(df)

61218

Filtramos los artículos sin body o con error

In [35]:
df=df[df['body']!='']
df=df[df['body']!='Please enable JS and disable any ad blocker']
df=df[df['body']!="We're sorry, we seem to be having some technical difficulties, but we don't want to lose you. Please report the issue here."]
df.reset_index(drop=True,inplace=True)
len(df)

60225

In [36]:
# group by 'body' column and count occurrences
counts = df.groupby('body').size().reset_index(name='count')

# select only rows where count is greater than 1
duplicates = counts.loc[counts['count'] > 1]

print(duplicates['count'].sum())
duplicates.sort_values('count', ascending=False)

364


,body,count
39418,Ride along with the Times contributor Tom Voel...,47
52987,Times journalists around the world bring you a...,29
2457,#SpeakingInDance is a weekly visual exploratio...,13
4930,"A traveler's guide to a city's most delicious,...",9
44895,The Best Chef in The World,8
...,...,...
16022,Evan Sung for The New York Times,2
14692,Dolly Faibyshev for The New York Times,2
14057,Danny Ghitis for The New York Times,2
12650,Consider that Honda's CR-V has been America's ...,2


# Combine texts

In [37]:
# concatenate the text columns in the order of preference and create a new column
df['full_text'] = df['headline'] + ' ' + df['caption'] + ' ' + df['abstract'] + ' ' + df['body']

# concatenate the text columns in the order of preference and create a new column
df['text_no_cap'] = df['headline'] + ' ' + df['abstract'] + ' ' + df['body']

# Transform labels to int

In [38]:
label_list=df['labels'].unique()
label_dict = {label: i for i, label in enumerate(label_list)}

def convert_label_to_int(x):
    return label_dict[x]

df=df.rename(columns={'labels': 'labels_text'})

In [41]:
df['labels']=df['labels_text'].apply(convert_label_to_int)
df

,labels_text,headline,abstract,caption,image_url,article_url,image_id,body,full_text,text_no_cap,labels
0,Movies,'Just Mercy' | Anatomy of a Scene,Destin Daniel Cretton narrates a sequence from...,"Jamie Foxx in ""Just Mercy."" Oscar attention fo...",https://static01.nyt.com/images/2020/01/11/art...,https://www.nytimes.com/video/movies/100000006...,1310e8-677d-5b7b-a19c-212d2dadc039,"""Hi, my name is Destin Cretton. I'm the direct...",'Just Mercy' | Anatomy of a Scene Jamie Foxx i...,'Just Mercy' | Anatomy of a Scene Destin Danie...,0
1,Sports,The Quiet Confidence of Naomi Osaka,"Osaka, the reigning Australian Open champion, ...","Naomi Osaka, the reigning Australian Open cham...",https://static01.nyt.com/images/2020/01/21/mul...,https://www.nytimes.com/2020/01/21/sports/tenn...,c0b4c69b-90fa-5f3f-8059-e3ce29b15570,"MELBOURNE, Australia -- As the reigning Austra...",The Quiet Confidence of Naomi Osaka Naomi Osak...,"The Quiet Confidence of Naomi Osaka Osaka, the...",1
2,Music,It's 'Salome.' But With Puppets.,A new staging in Austria looks all the more un...,"From left to right: John Daszak, Marlis Peters...",https://static01.nyt.com/images/2020/01/25/art...,https://www.nytimes.com/2020/01/21/arts/music/...,61e87a8f-7542-5f3a-8c44-08bc24f6207b,"From left to right: John Daszak, Marlis Peters...",It's 'Salome.' But With Puppets. From left to ...,It's 'Salome.' But With Puppets. A new staging...,2
3,Opinion,We Need a Law to Save Us From Dystopia,It's not too late. And it better be comprehens...,Hoan Ton-That testing the Clearview AI app.,https://static01.nyt.com/images/2020/01/21/opi...,https://www.nytimes.com/2020/01/21/opinion/fac...,a384a823-331e-59aa-ab59-dae56313edce,"Over the long weekend, my newsroom colleague K...",We Need a Law to Save Us From Dystopia Hoan To...,We Need a Law to Save Us From Dystopia It's no...,3
4,Media,Netflix Reports a Subscriber Bump as Disney Po...,"The streaming giant behind ""The Crown"" and ""Th...","Henry Cavill in a new Netflix series, ""The Wit...",https://static01.nyt.com/images/2020/01/21/bus...,https://www.nytimes.com/2020/01/21/business/me...,7ba85a5e-bcd6-51c3-b68c-c62b3766f744,"Reed Hastings, the chief executive of Netflix,...",Netflix Reports a Subscriber Bump as Disney Po...,Netflix Reports a Subscriber Bump as Disney Po...,4
...,...,...,...,...,...,...,...,...,...,...,...
60220,Global Business,"For Sushi Chain, Conveyor Belts Carry Profit","The Kura ""revolving sushi"" chain has succeeded...","Kura, a chain in Japan, relies on small staffs...",https://static01.nyt.com/images/2010/12/31/bus...,https://www.nytimes.com/2010/12/31/business/gl...,a3f8bf67-1938-5d2b-8d83-6e7edb80d5c0,"SAYAMA, Japan The Kura ""revolving sushi"" rest...","For Sushi Chain, Conveyor Belts Carry Profit K...","For Sushi Chain, Conveyor Belts Carry Profit T...",23
60221,Automobiles,Clouds Parted to a Forecast of Sunnier Days,"For the automotive industry, 2010 played out l...","DEBUTANTES Clockwise from far left, a few news...",https://static01.nyt.com/images/2011/01/02/aut...,https://www.nytimes.com/2011/01/02/automobiles...,75199d98-0c30-51b9-8efa-4783c078260e,THE last year in automobiles could have been m...,Clouds Parted to a Forecast of Sunnier Days DE...,Clouds Parted to a Forecast of Sunnier Days Fo...,22
60222,Automobiles,Jerry Garrett's Top 10 Cars of 2010,From a sleek European sedan to the last effort...,Aston Martin Rapide.,https://static01.nyt.com/images/2011/01/02/aut...,https://www.nytimes.com/2011/01/02/automobiles...,8af1e9f0-2fb8-57f1-9a10-25ca06743306,1. ASTON MARTIN RAPIDE The Rapide is something...,Jerry Garrett's Top 10 Cars of 2010 Aston Mart...,Jerry Garrett's Top 10 Cars of 2010 From a sle...,22
60223,Automobiles,Lawrence Ulrich's Top 10 Cars of 2010,From the world's best sports car to an unexpec...,Ferrari 458 Italia.,https://static01.nyt.com/images/2011/01/02/aut...,https://www.nytimes.com/2011/01/02/automobiles...,cfab59b0-2357-54b6-a469-7d5574d60e9c,1. FERRARI 458 ITALIA The world's best

# Dataset split

In [43]:
from sklearn.model_selection import train_test_split

# Split dataframe into train and test sets (80-20 split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED, stratify=df['labels'])

# Split test set into validation and test sets (50-50 split)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=SEED, stratify=test_df['labels'])

In [44]:
print(len(train_df), len(val_df), len(test_df))

train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

train_df.to_csv('../../data/splitted/train.csv', index=False)
val_df.to_csv('../../data/splitted/validation.csv', index=False)
test_df.to_csv('../../data/splitted/test.csv', index=False)

48180 6022 6023


In [20]:
from sklearn.model_selection import StratifiedKFold

y= df['labels']
X= df.drop(columns=['labels'])
skf=StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  print(f"Fold {i}:")
  print(f"  Train: index={train_index}  length={len(train_index)} ")
  print(f"  Test:  index={test_index} length={len(test_index)} ")

Fold 0:
  Train: index=[    0     2     3 ... 61215 61216 61217]  length=48974 
  Test:  index=[    1     4     9 ... 61203 61210 61213] length=12244 
Fold 1:
  Train: index=[    0     1     2 ... 61215 61216 61217]  length=48974 
  Test:  index=[    5    10    15 ... 61206 61208 61212] length=12244 
Fold 2:
  Train: index=[    1     2     3 ... 61214 61215 61216]  length=48974 
  Test:  index=[    0    27    29 ... 61197 61211 61217] length=12244 
Fold 3:
  Train: index=[    0     1     4 ... 61214 61215 61217]  length=48975 
  Test:  index=[    2     3     6 ... 61183 61209 61216] length=12243 
Fold 4:
  Train: index=[    0     1     2 ... 61213 61216 61217]  length=48975 
  Test:  index=[    7     8    22 ... 61207 61214 61215] length=12243 


# Change Nan from caption to ""

In [3]:
import pandas as pd
df_train=pd.read_csv('../../data/splitted/train.csv')
df_validation=pd.read_csv('../../data/splitted/validation.csv')
df_test=pd.read_csv('../../data/splitted/test.csv')

In [4]:
for column in df_train.columns:
    mask= pd.isna(df_train[column])
    if(sum(mask)>0):
        print(column)

caption


In [5]:
df_train.fillna(value="", inplace=True)
df_validation.fillna(value="", inplace=True)
df_test.fillna(value="", inplace=True)

In [9]:
mask=pd.isna(df_train['caption'])
df_train[mask]

,labels_text,headline,abstract,caption,image_url,article_url,image_id,body,full_text,text_no_cap,labels


In [11]:
df_train.to_csv('../../data/splitted/train.csv', index=False)
df_validation.to_csv('../../data/splitted/validation.csv', index=False)
df_test.to_csv('../../data/splitted/test.csv', index=False)